# Simulation counterbalancing vs. regression

In [1]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold, cross_val_score
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.feature_selection import f_classif
from skbold.preproc import ConfoundRegressor, MajorityUndersampler
%matplotlib inline

In [6]:
from counterbalance import CounterbalancedStratifiedSplit
from tqdm import tqdm_notebook

In [125]:
def generate_data(n_samp, n_feat, c_type, corr_cy, corr_Xy, signal_corr, confound_corr, verbose=False):
    
    if n_samp % 2 != 0:
        raise ValueError("Please select an even number of samples "
                         "(Makes things easier.)")

    y = np.repeat([0, 1], repeats=n_samp/2)
    
    if c_type == 'categorical':
        to_roll = int((n_samp / 4) * (1 - corr_cy))
        c = np.roll(y, to_roll)
    else:
        noise_factor = 100
        c = y + np.random.randn(n_samp) * noise_factor
        corr = pearsonr(c, y)[0]
        while np.abs(corr - corr_cy) > 0.01:
            noise_factor -= 0.01
            c = y + np.random.randn(n_samp) * noise_factor
            corr = pearsonr(c, y)[0]        
    
    signal_y, signal_c = 1, 1 / corr_cy
    noise_level = 1
    
    X = np.zeros((n_samp, n_feat))
    total_corr = np.zeros(n_feat)
    pcorr_noise = np.zeros(n_feat)
    pcorr_signal = np.zeros(n_feat)
    
    iterator = tqdm_notebook(np.arange(n_feat)) if verbose else np.arange(n_feat)
    
    for i in iterator:
        
        tries = 0
        
        while True:
            
            this_X = np.random.randn(y.size) * noise_level + signal_y * y + signal_c * c
            this_total_corr = pearsonr(this_X, y)[0]
            
            if np.abs(this_total_corr - corr_Xy) > 0.01:
                if corr_Xy > this_total_corr:
                    noise_level -= 0.05
                else:
                    noise_level += 0.05

                continue
            else:
                residuals_cx = this_X - c[:, np.newaxis].dot(np.linalg.lstsq(c[:, np.newaxis], this_X)[0])
                residuals_cy = y - c[:, np.newaxis].dot(np.linalg.lstsq(c[:, np.newaxis], y)[0])
                this_pcorr_signal = pearsonr(residuals_cx, residuals_cy)[0]
                this_pcorr_noise = this_total_corr - this_pcorr_signal

                signal_ready = np.abs(this_pcorr_signal - signal_corr) < 0.01
                confound_ready = np.abs(this_pcorr_noise - confound_corr) < 0.01
                
                if signal_ready and confound_ready:
                    X[:, i] = this_X
                    total_corr[i] = this_total_corr
                    pcorr_signal[i] = this_pcorr_signal
                    pcorr_noise[i] = this_pcorr_noise
                    break
                else:
                    tries += 1
                
                if tries > 1000:
                    # Try adjusting relative signal strengths
                    if this_pcorr_noise > confound_corr:
                        signal_y += 0.01
                        signal_c -= 0.01
                    else:
                        signal_y -= 0.01
                        signal_c += 0.01
    
    if verbose:
        print("Corr(c, y): %.3f" % pearsonr(c, y)[0])    
        print("Corr(X, y): %.3f" % total_corr.mean())
        print("Partial corr(X_confound, y): %.3f" % pcorr_noise.mean())
        print("Partial corr(X_signal, y): %.3f" % pcorr_signal.mean())
                    
    return X, y, c

In [126]:
confound_corr = 0.8
X, y, c = generate_data(n_samp=120, n_feat=5, c_type='categorical', corr_cy=0.5, corr_Xy=0.5,
                        signal_corr=0.5-confound_corr, confound_corr=confound_corr, verbose=True)

0.542960500344
0.57959702369
0.592550009171
0.605102834737
0.611713235228
0.455790578251
0.516391525522
0.541429740202
0.539779237409
0.539323018603
0.474007265312
0.498861074639
0.476293368286
0.533255130594
0.597975278036
0.495487176153
0.522162882257
0.411600180423
0.544503569271
0.512167155624
0.565513204088
0.37915438192
0.573710864463
0.497925061645
0.521677813968
0.45450520645
0.456918149225
0.470931959865
0.500425782024
0.471492283414
0.609102754986
0.533721784239
0.469238324347
0.45356668263
0.449866044039
0.592708062252
0.514395183843
0.51482894924
0.460054357934
0.428720032447
0.543870457769
0.478195393255
0.346280696596
0.61421508705
0.556524567839
0.367151766203
0.597296294355
0.580660205122
0.548284817094
0.464780596862
0.465765069365
0.470841049191
0.576997969987
0.522913266284
0.495975009118
0.535600647896
0.50626932073
0.522357863435
0.391597766508
0.605598100446
0.376538599538
0.409072699304
0.529148561563
0.348584924244
0.495520408964
0.54396941221
0.498008610842
0.5

0.543901016157
0.51466733541
0.491744495283
0.585265341038
0.510386760952
0.517437093263
0.436562523211
0.412371269323
0.469781025016
0.444422632659
0.495028117487
0.389073098375
0.464167000982
0.415744916332
0.498180268131
0.371202326825
0.484858813814
0.447619409772
0.560371548501
0.567684489964
0.47746578243
0.434788976347
0.537689291308
0.625241622334
0.49090450255
0.419403431221
0.503694531278
0.513512883519
0.513601511818
0.483487581723
0.539427462197
0.461414999264
0.586864118064
0.568011224385
0.478445704671
0.502500849574
0.555267216701
0.460368441491
0.36147327497
0.54826796545
0.454767313759
0.462464521155
0.586026503411
0.477884492821
0.400295172514
0.547707825231
0.616669308125
0.541155381004
0.468135831427
0.560565868382
0.561740888605
0.476396617952
0.539528483461
0.568014397955
0.555098740009
0.577920482387
0.488889125292
0.404554710633
0.402705277872
0.539482401521
0.472724421908
0.62732637354
0.449713423477
0.535081135368
0.437407219147
0.531610985097
0.536319709206
0

0.52790250362
0.542214296922
0.501200619751
0.402082766463
0.519689501426
0.491628889727
0.579212532238
0.375193107508
0.329293418539
0.513668222909
0.492535315058
0.482647238383
0.398192920256
0.542365527739
0.512015472812
0.496605616182
0.520123160081
0.391346547517
0.466688355542
0.519941888413
0.53882486284
0.469498172661
0.40116971034
0.529519502079
0.610006636365
0.512455985188
0.455959154908
0.29584583771
0.588316612546
0.402453810393
0.483326344641
0.51626121289
0.457806439903
0.394230308935
0.586360762762
0.51815343719
0.445354450026
0.437316612819
0.482352316504
0.509307857691
0.472044381877
0.536479570404
0.561872002772
0.53022752139
0.460600612914
0.504780478578
0.552867526326
0.455996017814
0.467623860841
0.666180051923
0.445375373065
0.482639004875
0.519240976343
0.46890760881
0.56619631092
0.461253651471
0.536303857053
0.472522544836
0.581608257004
0.56152922115
0.48931238998
0.410942950736
0.514951155324
0.456379036121
0.586761257698
0.544162427186
0.390551655017
0.6013

0.58908719643
0.510386849178
0.44236971886
0.462785166528
0.512035321796
0.389598835297
0.490053262578
0.524794807455
0.521255662727
0.366940153184
0.460146740425
0.422307253273
0.507706505637
0.563073334031
0.524047948823
0.421642782254
0.445239839879
0.42174017627
0.438964005698
0.441421987834
0.601837456947
0.510158976224
0.612672115095
0.580807697692
0.52185811635
0.458141883521
0.527763732679
0.450611648643
0.470672787997
0.427402892932
0.422113372596
0.513815432645
0.451432076685
0.457269406306
0.591288272362
0.598799762029
0.397673576684
0.52203500006
0.533651384524
0.554630823401
0.525672846611
0.531879965289
0.450099593633
0.469354432704
0.495623176343
0.511058047198
0.530980552434
0.528229521018
0.485795663462
0.509052387776
0.38413579289
0.447924558931
0.51874009777
0.576876220716
0.60791803907
0.509748844805
0.439364447627
0.337103334071
0.445396374934
0.422318526681
0.481726355505
0.552139336184
0.54740789047
0.506886107299
0.367003183033
0.533856776459
0.444871487773
0.55

0.47158697693
0.501478207665
0.397664397107
0.547258607776
0.54214486568
0.46349633529
0.421862610022
0.609862051891
0.495758069029
0.479722162022
0.525975355014
0.458344124033
0.510637952153
0.559428047244
0.524367370126
0.45349499806
0.488413241962
0.465540900499
0.543239992005
0.562500380105
0.536554317357
0.555637425153
0.473019407809
0.481098582998
0.478774465629
0.499305764648
0.511256768171
0.532983534402
0.530557788697
0.519225297839
0.474787516718
0.514961014657
0.427985789361
0.49393401456
0.386329548361
0.487392652465
0.482935664689
0.594753516328
0.582387677428
0.405872142502
0.51811505462
0.526943830566
0.595085022354
0.527018406855
0.38258900802
0.499195663241
0.488251330288
0.530391357207
0.486830385235
0.435668688405
0.432065378615
0.41569740804
0.464111769538
0.506346533873
0.417500455232
0.462851015257
0.521732090172
0.554309652825
0.533888121028
0.517398027312
0.607552908103
0.53972590206
0.483516572677
0.455733378249
0.474598843038
0.551635035408
0.572015585939
0.45

...

0.107690740443
0.0584163180113
-0.141472860534
0.00897229066339
-0.0644510803166
0.0371610379048
0.00726328750859
-0.0915050277237
0.0260411946512
-0.0621729225679
-0.0931072453129
-0.00629274957382
0.122640071946
0.0755694355939
0.00278413141373
-0.00778165447407
-0.0115728367844
0.209806568859
0.0352312669
-0.0777818734226
-0.108377919412
-0.0233523315613
-0.0143431867135
-0.0620327582593
0.0537606983531
-0.118781400775
-0.0594291188187
-0.0955732470235
-0.18757165572
-0.1541479011
-0.0189768715399
-0.0594739781158
0.118437833823
-0.14275753729
0.0153403779594
0.0116979394842
0.0507110811441
0.0900936018041
-0.184700301274
-0.0111089451308
0.0185346111731
0.117968865311
-0.0548592388163
0.058620011311
0.115439340468
-0.0227074139182
0.103930737202
0.0622344205242
-0.102366184691
0.116913086689
0.00395916288903
0.0490472985646
0.03063429743
-0.103332596262
0.134155298344
-0.00201992936202
-0.00411111985419
0.0234039481487
0.0138445283399
0.0165523459393
0.0177889678289
0.0312638723261

0.0383617088519
-0.0945754754551
-0.0270029257954
-0.106532860826
-0.0820045055576
0.132043210443
0.017644410301
0.0280893509536
0.0816407248681
-0.0617106945973
-0.0166949244802
-0.00495610295082
-0.166861500632
0.146172129235
-0.132026327107
0.0603966283128
-0.0710489448712
-0.103554601599
-0.0281259614891
-0.0665631054413
0.12270549694
-0.137739364256
-0.132700887253
-0.0848861724206
0.00919042161024
0.0456460142149
0.0850569555912
-0.0516006517481
-0.0483697562643
0.0408086067138
-0.197709289691
0.0185173760205
-0.0365163903395
-0.0508312485767
0.144915766677
-0.0122217263198
-0.119637395205
-0.0559401928874
0.0616566161591
-0.0166766420232
-0.176846505194
-0.0275572008899
-0.241633297149
-0.0967055061471
0.013257387403
-0.151939174033
-0.00149101217223
0.0163314597465
-0.105090559307
0.143595246118
-0.0371275901503
-0.0664147373661
0.0342482218924
0.0184234066885
0.0116681777206
-0.0169610373147
-0.182430635804
-0.0254454060432
0.0566853702116
-0.0111456179116
0.0662136851897
-0.0

-0.0951785927922
-0.0344333390416
-0.17150580153
-0.0816873827028
-0.249234192582
-0.110627395423
-0.00784099612249
0.165475782489
-0.194316694254
-0.0856972220358
0.0672224724587
0.0370740835819
0.085247281243
0.0393097237858
-0.093585506106
0.108295519771
-0.189275177001
0.0391628274543
-0.0228582544701
-0.0410950966852
0.0823461495913
-0.186641852158
0.0147859938258
0.148434348033
0.041657195396
0.00867052829713
-0.0626858621118
0.141936871631
0.139296554657
0.0827447377944
-0.233996535738
0.0378910364286
0.0382044356878
0.00463769492744
-0.0683114619613
-0.119656325229
0.148956973212
-0.00539011838871
-0.0214484013778
0.0862388303879
-0.07048999457
-0.0719580780115
0.0824852131222
-0.0267675340162
0.1737573151
-0.097827804819
0.0548526664799
-0.0862088440024
0.0327065436206
-0.0717519415191
-0.119848902187
-0.0261507428845
0.137549766678
0.143134362294
0.0245179079977
-0.0799176321218
0.144441357343
-0.0620688065918
0.0642693321878
0.184086308523
-0.162690225955
-0.0535934383975
0.

-0.0114041635264
0.175507146479
-0.02821327316
0.0107075318876
-0.136615549137
-0.0506604885643
-0.0127398692376
0.104037466156
0.192550775279
-0.149055411797
-0.0132061479464
0.0461931048952
0.0356830293656
-0.0382403884567
0.15342183672
-0.0738958641859
-0.00822645956384
-0.162290261969
-0.038390652344
-0.0838846464045
-0.00290486592548
0.0887856826662
0.0374277249563
0.106747650463
-0.00731472760954
-0.13148984544
0.165506285146
-0.0432386887587
0.0646627163172
0.0814517280122
-0.123533650301
-0.156012176494
-0.124008192253
0.117904816396
-0.198782527333
0.0381494314112
-0.0373773348054
-0.0439233006244
0.0628626339602
0.120768388486
-0.0987940217241
0.0272382940207
0.05671062226
0.0705922149066
0.0582961659172
-0.20431868415
0.0400736262526
0.0190391505148
-0.0905183642821
0.00205203868515
-0.110828037834
-0.016019968691
0.162664781912
0.0411317799
-0.0470961226429
0.0267083508283
0.0527953218262
-0.0262803558612
0.0524577332042
0.137676065822
0.0906074637259
0.0544405686459
0.0236

KeyboardInterrupt: 

In [115]:
from copy import deepcopy

pipeline = [
    ('scaler', StandardScaler()),
    ('clf', SVC(kernel='linear'))
    ]

n_splits = 10
permutations = 10

results = {'confound_corr': [],
           'method': [],
           'score': []}

for perm in np.arange(permutations):
    print(perm)
    for confound_corr in np.linspace(0.00, 0.3, 11):
    
        X, y, c = generate_data(n_samp=240, n_feat=5, c_type='categorical', corr_cy=0.5, corr_Xy=0.3+confound_corr,
                                signal_corr=0.3, confound_corr=confound_corr, verbose=False)
    
        skf = StratifiedKFold(n_splits=n_splits)
        scores = np.zeros(n_splits)
        this_pipe = Pipeline(pipeline)
    
        for i, (train_idx, test_idx) in enumerate(skf.split(X, y)):
            X_train, y_train = X[train_idx], y[train_idx]
            X_test, y_test = X[test_idx], y[test_idx]
            this_pipe.fit(X_train, y_train)
            scores[i] = this_pipe.score(X_test, y_test)
            results['confound_corr'].append(confound_corr)
            results['method'].append('none')
            results['score'].append(scores[i])

        skf = StratifiedKFold(n_splits=n_splits)
        scores = np.zeros(n_splits)
        for i, (train_idx, test_idx) in enumerate(skf.split(X, y)):
            cr = ConfoundRegressor(confound=c, fit_idx=train_idx, cross_validate=True)
            
            this_pipe = deepcopy(pipeline)
            this_pipe.insert(0, ('regress', cr))
            this_pipe = Pipeline(this_pipe)
            X_train, y_train = X[train_idx], y[train_idx]
            X_test, y_test = X[test_idx], y[test_idx]
            this_pipe.fit(X_train, y_train)
            scores[i] = this_pipe.score(X_test, y_test)

            results['confound_corr'].append(confound_corr)
            results['method'].append('regress')
            results['score'].append(scores[i])

        skf = CounterbalancedStratifiedSplit(X, y, c, n_splits=n_splits, c_type='categorical', verbose=True)
        try:
            skf.check_counterbalance_and_subsample()
        except ValueError:
            results['confound_corr'].extend([confound_corr] * n_splits)
            results['method'].extend(['cb'] * n_splits)
            results['score'].extend([0] * n_splits)
            continue

        X, y, c = X[skf.subsample_idx], y[skf.subsample_idx], c[skf.subsample_idx]    
        scores = np.zeros(n_splits)
        this_pipe = Pipeline(pipeline)
        for i, (train_idx, test_idx) in enumerate(skf.split(X, y)):
            X_train, y_train = X[train_idx], y[train_idx]
            X_test, y_test = X[test_idx], y[test_idx]
            this_pipe.fit(X_train, y_train)
            scores[i] = this_pipe.score(X_test, y_test)
            results['confound_corr'].append(confound_corr)
            results['method'].append('cb')
            results['score'].append(scores[i])
    
df = pd.DataFrame(results)
g = sns.factorplot(x='confound_corr', y='score', hue='method', data=df, size=5, aspect=2, legend_out=False,
                   palette="muted", legend=True)
plt.axhline(0.5, c='black', ls='--')
plt.show()

0
Picking seed 6434233
Size of y after subsampling: 120 (50.0 percent reduction in samples)
Picking seed 99171
Size of y after subsampling: 120 (50.0 percent reduction in samples)
Picking seed 1218624
Size of y after subsampling: 120 (50.0 percent reduction in samples)
Picking seed 1486136
Size of y after subsampling: 120 (50.0 percent reduction in samples)
Picking seed 1190649
Size of y after subsampling: 120 (50.0 percent reduction in samples)
Picking seed 9964170
Size of y after subsampling: 120 (50.0 percent reduction in samples)
Picking seed 7110254
Size of y after subsampling: 120 (50.0 percent reduction in samples)
Picking seed 9151817
Size of y after subsampling: 120 (50.0 percent reduction in samples)
Picking seed 9079018
Size of y after subsampling: 120 (50.0 percent reduction in samples)


KeyboardInterrupt: 